# Basic Alhazen Agent / Chatbot.

> Building a simple Gradio application to allow the user to chat to Alhazen. 

In [ ]:
#| default_exp apps.chat

In [1]:
#| export

from alhazen.core import OllamaRunner, PromptTemplateRegistry
from alhazen.schema_sqla import *
from alhazen.tools.basic import AddCollectionFromEPMCTool, DeleteCollectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import MetadataExtractionTool, MetadataExtractionWithRAGTool 
from alhazen.toolkit import AlhazenToolkit
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.chat_models import ChatOllama
from langchain.pydantic_v1 import BaseModel
from langchain.tools.render import render_text_description, render_text_description_and_args
from langchain.agents.output_parsers.json import JSONAgentOutputParser

import gradio as gr
import os
import pandas as pd
from pathlib import Path
import re
import requests
from typing import Dict, List, Optional, Sequence, Union
from uuid import UUID

/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#| export 

class AlhazenAgentChatBot:
    '''Provide access to an Alhazen agent via a simple chat interface with Gradio.'''

    def __init__(self):

        if os.environ.get('ALHAZEN_DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['ALHAZEN_DB_NAME']

        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']

        self.db = Ceifns_LiteratureDb(loc=loc, name=db_name)
        self.ollr = OllamaRunner('mixtral')
        self.llm  = self.ollr.llm

        tk = AlhazenToolkit(db=self.db, ollr=self.ollr)
        self.tools = tk.get_tools()

        pts = PromptTemplateRegistry()
        pts.load_prompts_from_yaml('alhazen_base.yaml')

        self.prompt = pts.get_prompt_template('structured chat agent').generate_chat_prompt_template()
        
        agent = create_structured_chat_agent(self.llm, self.tools, self.prompt)

        self.agent_executor = AgentExecutor(agent=agent, 
                                            tools=self.tools,
                                            verbose=True, 
                                            handle_parsing_errors=True)

    # We need to add these input/output schemas because the current AgentExecutor
    # is lacking in schemas.
    class ChatRequest(BaseModel):
        question: str
        chat_history: Optional[List[Dict[str, str]]]

    class Input(BaseModel):
        input: str

    class Output(BaseModel):
        output: str

    def run_gradio(self):

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)
            
        def clear_chat(history):
            return []

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            response = self.agent_executor.invoke({"input": prompt}, config={'callbacks': [ConsoleCallbackHandler()]})
            print('RESPONSE: %s'%(str(response)))
            history[-1][1] = str(response.get('output'))
            print('WHOLE HISTORY: %s'%(history))
            return history

        with gr.Blocks() as demo:
            with gr.Tab("Chat"):
                chatbot = gr.Chatbot(
                    [],
                    elem_id="chatbot",
                    bubble_full_width=False,
                    #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
                )
                with gr.Row():
                    txt = gr.Textbox(
                        scale=4,
                        show_label=False,
                        placeholder="Enter text and press enter.",
                        container=False,
                    )
                    clear_btn = gr.Button("Clear")
            with gr.Tab("Dashboard"):
                with gr.Row():
                    doc_text = gr.HTML(label="File Contents")
            
            txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
            txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)                
            clear_btn.click(clear_chat, [], [chatbot], queue=False)
                
        demo.queue()
        demo.launch()

In [11]:
db_name = 'em_tech' 
loc = '/Users/gully.burns/alhazen'

db = Ceifns_LiteratureDb(loc=loc, name=db_name)
ollr = OllamaRunner('mixtral')
llm  = ollr.llm

cb = AlhazenAgentChatBot()
agent_response = cb.agent_executor({"input": 'Hello'})
print(agent_response.get('output'))

In [ ]:
print(cb.prompt)

In [21]:
os.environ['ALHAZEN_DB_NAME'] = 'em_tech' 
os.environ['LOCAL_FILE_PATH'] = '/Users/gully.burns/alhazen'
cb = AlhazenAgentChatBot()
cb.run_gradio()

/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/utils.py:827: UserWarning: Expected 1 arguments for function <function AlhazenAgentChatBot.run_gradio.<locals>.clear_chat at 0x41f80f060>, received 0.
  warnings.warn(
/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/utils.py:831: UserWarning: Expected at least 1 arguments for function <function AlhazenAgentChatBot.run_gradio.<locals>.clear_chat at 0x41f80f060>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
 Thought: The user is saying hello, so I should respond with a friendly greeting.
Action:
```
{
  "action": "Final Answer",
  "action_input": {
    "greeting": "Hello! Welcome to the scientific paper analysis system. How can I assist you today?"
  }
}
```

> Finished chain.
RESPONSE: {'input': 'Hello', 'output': {'greeting': 'Hello! Welcome to the scientific paper analysis system. How can I assist you today?'}}
WHOLE HISTORY: [['Hello', "{'greeting': 'Hello! Welcome to the scientific paper analysis system. How can I assist you today?'}"]]


> Entering new AgentExecutor chain...
 Thought: The user wants to know what this system can do. I will list the capabilities of the system by providing a JSON blob with the "action" set to "Final Answer" and the "action\_input" set to a description of the system's abilities.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "This system can perform various actions related to scientific research.

Traceback (most recent call last):
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/blocks.py", line 1522, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packag



> Entering new AgentExecutor chain...


Traceback (most recent call last):
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/langchain/agents/output_parsers/json.py", line 43, in parse
    response = parse_json_markdown(text)
               ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/site-packages/langchain/output_parsers/json.py", line 145, in parse_json_markdown
    parsed = parser(json_str)
             ^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gully.burns/miniconda3/envs/alhazen/lib/python3.11/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, id